In [6]:
import numpy as np
import pandas as pd 
import random
import pickle
n=3

In [7]:
class State:
    def __init__(self,p1,p2) -> None:
        self.board=np.zeros((n,n))
        self.p1=p1
        self.p2=p2
        self.isEnd=False
        self.boardHash=None
        self.playerSymbol=1
        #we Start with Player 1
    def getHash(self):
        self.boardHash=str(self.board.reshape(n*n))
        print(self.boardHash)
        return self.boardHash
    def winner(self):
        for row in range(3):
            if self.board[row][0] == self.board[row][1] == self.board[row][2] == 1:
                self.isEnd= True
                return 1
            if self.board[row][0] == self.board[row][1] == self.board[row][2] == -1:
                self.isEnd= True
                return -1

        # Check columns
        for col in range(3):
            if self.board[0][col] == self.board[1][col] == self.board[2][col] == 1:
                self.isEnd= True
                return 1
            if self.board[0][col] == self.board[1][col] == self.board[2][col] == -1:
                self.isEnd= True
                return -1

        # Check diagonals
        if self.board[0][0] == self.board[1][1] ==self.board[2][2] == 1:
            self.isEnd= True
            return 1
        if self.board[0][0] == self.board[1][1] ==self.board[2][2] == -1:
            self.isEnd= True
            return -1
        if self.board[0][2] == self.board[1][1] == self.board[2][0] == 1:
            self.isEnd= True
            return 1
        if self.board[0][2] == self.board[1][1] == self.board[2][0] == -1:
            self.isEnd= True
            return -1

        if len(self.availablePositions())==0:
            self.isEnd=True
            return 0
        self.isEnd=False
        return None
    def availablePositions(self):
        positions=[]
        for i in range(n):
            for j in range(n):
                if self.board[i][j]==0:
                    positions.append((i,j))
        return positions
    def updateState(self,position):
        self.board[position]=self.playerSymbol
        self.playerSymbol= -1 if self.playerSymbol==1 else 1
    def giveReward(self):
        result=self.winner()
        if result==1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result==-1:
            self.p2.feedReward(1)
            self.p1.feedReward(0)
        else:
            self.p2.feedReward(0.2)
            self.p1.feedReward(0.5)
    
    def reset(self):
        self.board=np.zeros((n,n))
        self.isEnd=False
        self.boardHash=None
        self.playerSymbol=1
    

    def play(self,rounds=100):
        for i in range(rounds):
            if i%1000==0:
                print("Round {}".format(i))
            
            while not self.isEnd:
                positions=self.availablePositions()
                p1_action=self.p1.chooseAction(positions,self.board,self.playerSymbol)
                self.updateState(p1_action)
                board_hash=self.getHash()
                self.p1.addState(board_hash)

                win=self.winner()
                if win is not None:
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break
                else:
                    positions=self.availablePositions()
                    p2_action=self.p2.chooseAction(positions,self.board,self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash=self.getHash()
                    self.p2.addState(board_hash)
                    win=self.winner()
                    if win is not None:
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
    def showboard(self):
    #p1 x p2 O
        for i in range(n):
            print("-------------------")
            out=" | "
            for j in range(n):
                if self.board[i,j]==1:
                    token='X'
                if self.board[i,j]==-1:
                    token='O'
                if self.board[i,j]==0:
                    token=' '
                out+=token+"  |  "
            print(out)
        print("----------------")

    def play2(self):
        while not self.isEnd:
            positions=self.availablePositions()
            p1_action=self.p1.chooseAction(positions,self.board,self.playerSymbol)
            self.updateState(p1_action)
            print(self.board)
            #showboard()
            win=self.winner()
            if win is not None:
                if win==1:
                    print(self.p1.name,"Winnner")
                else:
                    print("tie")
                self.reset()
                break
            else:
                positions=self.availablePositions()
                p2_action=self.p2.chooseAction(positions)
                self.updateState(p2_action)
                print(self.board)

                #self.showboard()
                win=self.winner()
                if win is not None:
                    if win == -1:
                        print(self.p2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break

            



In [8]:
class Player:
    def __init__(self,name,exp_rate=0.3):
        self.name=name
        self.states=[]
        self.lr=0.2
        self.exp_rate=exp_rate
        self.decay_gamma=0.9
        self.states_value={}
    def gethash(self,board):
        boardHash=str(board.reshape(n*n))
        return boardHash
    def chooseAction(self,positions,current_board,symbol):
        if np.random.uniform(0,1)<=self.exp_rate:
            idx=np.random.choice(len(positions))
            action=positions[idx]
        else:
            value_max=-999
            for p in positions:
                next_board=current_board.copy()
                next_board[p]=symbol
                next_boardHash=self.gethash(next_board)
                value=0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                if value>=value_max:
                    value_max=value
                    action=p
        return action
    def addState(self,state):
        self.states.append(state)

    def feedReward(self,reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st]=0
            self.states_value[st]+=self.lr*(self.decay_gamma*reward-self.states_value[st])
            reward=self.states_value[st]
    
    def reset(self):
        self.states=[]
    def savePolicy(self):
        fw=open('policy_'+str(self.name),'wb')
        pickle.dump(self.states_value,fw)
        fw.close()
    def loadPolicy(self,file):
        fr=open(file,'rb')
        self.states_value=pickle.load(fr)
        fr.close()


In [9]:
class HumanPlayer:
    def __init__(self,name):
        self.name=name
    def chooseAction(self,positions):
        while True:
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions:
                return action
    def addState(self,state):
        pass
    def feedReward(self,reward):
        pass
    def reset(self):
        pass
    

In [10]:
if __name__=="__main__":
    p1=Player("p1")    
    p1.loadPolicy("policy_p1")
    h1=HumanPlayer("bar")
    s1=State(p1,h1)
    s1.play2()

 


[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 1.  0. -1.]
 [ 0.  0.  0.]]
[[ 0.  0.  0.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]]
[[-1.  0.  0.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]]
[[-1.  0.  1.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]]
[[-1. -1.  1.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]]
[[-1. -1.  1.]
 [ 1.  0. -1.]
 [ 0.  1.  1.]]
[[-1. -1.  1.]
 [ 1.  0. -1.]
 [-1.  1.  1.]]
[[-1. -1.  1.]
 [ 1.  1. -1.]
 [-1.  1.  1.]]
tie
